In [ ]:
import torch
import random
import evaluate
import datasets
import numpy as np
import pandas as pd
import transformers
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
# Get dataset
dataset = load_dataset("cnn_dailymail")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 11490
    })
})

In [5]:
# Show some examples picked randomly in the dataset
def show_random_elements(dataset, num_examples=5):
    # Ensure that num_examples is not greater than the number of examples in the dataset
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    
    # Initialize an empty list to store the randomly picked indices
    picks = []
    
    # Loop to randomly pick num_examples elements from the dataset
    for _ in range(num_examples):
        # Randomly select an index within the range of the dataset
        pick = random.randint(0, len(dataset)-1)
        
        # Check if the index has already been picked to avoid duplication
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        
        # Add the picked index to the list of picks
        picks.append(pick)
    
    # Create a DataFrame containing the randomly picked examples from the dataset
    df = pd.DataFrame(dataset[picks])
    
    # Transform class label indices into their corresponding names (if applicable)
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    
    # Display the DataFrame containing the random examples
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

In [23]:
# Define Model and Tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"

model_checkpoint = "facebook/bart-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

NameError: name 'torch' is not defined

In [8]:
# Prefix the input with a prompt so BART knows this is a summarization task
prefix = "summarize: "

def preprocess_function(examples):
    # Add a prefix to each document in the "article" column
    # The prefix might be useful for the model to understand that it's dealing with document summaries
    inputs = [prefix + doc for doc in examples["article"]]
    
    # Tokenize the modified documents using the tokenizer
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Tokenize the "highlights" column to create labels for the model
    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

    # Include the tokenized labels (input_ids) in the model_inputs dictionary
    # This is necessary for training the model with the token-level classification objective
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [9]:
# Apply the function on all the elements in dataset(training, testing, and validation) data will be preprocessed in one single command
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [10]:
# To dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

In [1]:
# Load a evaluation method (ROUGE) with the Evaluate library
rouge = evaluate.load("rouge")

In [13]:
def compute_metrics(eval_pred):
    # Unpack the tuple containing predictions and labels
    predictions, labels = eval_pred
    
    # Decode the predicted sequences, skipping special tokens
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace label values of -100 (masked tokens) with pad token id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Decode the labels, skipping special tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores between predictions and labels
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Compute average length of generated predictions
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Round the metrics to 4 decimal places and return as a dictionary
    return {k: round(v, 4) for k, v in result.items()}

In [19]:
batch_size = 8
num_train_epochs = 8

# Show the training loss with every epoch
logging_steps = len(tokenized_dataset["train"]) // batch_size
model_name = model_checkpoint

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-CNN",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps
)

In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

NameError: name 'model' is not defined

In [ ]:
trainer.train()